In [1]:
%matplotlib inline

# include modules to the path
import sys, os
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parent_dir)
sys.path.append(os.path.join(parent_dir, 'session'))

from pack import pack, write_units, write_spiking_metrics, write_spatial_metrics,\
                    write_best_match_rotation

import os
import numpy as np
import h5py, json
import matplotlib.pyplot as plt
import scipy.ndimage as ndi
from scipy import signal
from session.utils import get_sessions_list, get_sampling_rate, cleaned_epochs
from session.adapters import load_clu_res, H5NAMES, create_dataset
from head_direction import head_direction
#from spatial import place_field_2D, map_stats, get_field_patches
#from spatial import bins2meters, cart2pol, pol2cart
from spiketrain import instantaneous_rate, spike_idxs
from spiking_metrics import mean_firing_rate, isi_cv, isi_fano

from session.lists import processed_008228, processed_008229
from session.sessions import selected_009265

In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

## Set session paths and steps to perform

In [3]:
EPOCH_NAMES = ('Original', 'Conflict', 'Control', 'All')

source = '/home/sobolev/nevermind/Andrey/data'
#source = '/home/sobolev/nevermind/Michael/FreeBehaving/SIT_sessions/'
# GPU-XXL
#source = '/mnt/nevermind.data-share/ag-grothe/Andrey/data'

# test session
#filebase = '008229_hippoSIT_2022-05-16_20-36-44'
#filebase = '64_prSIT_2023-02-27_15-06-02'
#filebase = '009266_hippoSIT_2023-04-24_16-56-55'
#animal = filebase.split('_')[0]
#sessionpath = os.path.join(source, animal, filebase)
#h5name  = os.path.join(source, animal, filebase, filebase + '.h5')
#sessionpath = os.path.join(source, filebase)
#h5name  = os.path.join(source, filebase, filebase + '.h5')

#sessions = get_sessions_list(os.path.join(source, animal), animal)
#sessions = get_sessions_list(source, animal)
#sessions = [filebase]
#sessions = processed_008229
sessions = [
#'009266_hippoSIT_2023-04-17_17-04-17',  # ch17, 20 + 55 correction, 5067 events. Showcase for N2 / N3 mod in target
#'009266_hippoSIT_2023-04-18_10-10-37',  # ch17, 10 + 55 correction, 5682 events
#'009266_hippoSIT_2023-04-18_17-03-10',  # ch17, 6 + 55 correction, 5494 events: FIXME very weird 1-2nd in target, find out
#'009266_hippoSIT_2023-04-19_10-33-51',  # ch17, 4 + 55 correction, 6424 events: very weird 1-2nd in target, find out
#'009266_hippoSIT_2023-04-20_08-57-39',  # ch1, 1 + 55 correction, 6424 events. Showcase for N2 / N3 mod in target
#'009266_hippoSIT_2023-04-24_16-56-55',  # ch17, 5 + 55* correction, 6165 events, frequency
#'009266_hippoSIT_2023-04-26_08-20-17',  # ch17, 12 + 55* correction, 6095 events, duration - showcase for N2 
#'009266_hippoSIT_2023-05-02_12-22-14',  # ch20, 10 + 55 correction, 5976 events, FIXME very weird 1-2nd in target, find out
#'009266_hippoSIT_2023-05-04_09-11-06',  # ch17, 5 + 55* correction, 4487 events, coma session with baseline AEPs
#'009266_hippoSIT_2023-05-04_19-47-15',
    # PPC
#'009266_hippoSIT_2023-04-20_15-24-14',  # A1 ch20, PPC ch32, 60 + 55 correction, 5612 events
#'009266_hippoSIT_2023-04-21_08-43-00',  # A1 ch20, PPC ch32, 72 + 55 correction, 6282 events
#'009266_hippoSIT_2023-04-21_13-12-31',  # A1 ch20, PPC ch32, 72 + 55 correction, 6041 events
#'009266_hippoSIT_2023-04-24_10-08-11',  # A1 ch20, PPC ch40, 80 + 55 correction, 5579 events
    # HPC
# '009266_hippoSIT_2023-05-22_09-27-22',
# '009266_hippoSIT_2023-05-23_09-18-05',
# '009266_hippoSIT_2023-05-25_15-55-57',
# '009266_hippoSIT_2023-06-14_08-21-23',
# '009266_hippoSIT_2023-06-19_08-58-35',
    
#'009265_hippoSIT_2023-03-08_18-10-07',
#'009265_hippoSIT_2023-03-09_20-03-08',
#'009265_hippoSIT_2023-03-10_09-57-34',
'60_SIT_2023-11-24_17-09-10',
]

# FIXME move occupancy outside units
# FIXME Move 'processed' into H5NAMES

steps = {
    'write_trajectory': True,        # Packing to HDF5 (trajectory, trials, sound events)
    'write_units': True,             # Loading units (spikes times, trajectory indices, instantaneous rate)
    'write_spiking_metrics': True,  # Spiking metrics (mean firing rate, ISI metrics etc.)
    'write_spatial_metrics': False,  # Spatial metrics
    'write_best_mat_rot': False,     # Best match rotation
}

sessions

['60_SIT_2023-11-24_17-09-10']

In [21]:
get_sessions_list(os.path.join(source, animal), animal)[:3]

['009266_hippoSIT_2023-02-28_19-53-56',
 '009266_hippoSIT_2023-03-01_17-53-11',
 '009266_hippoSIT_2023-03-06_15-10-36']

In [4]:
# execute
for session in sessions:
    animal = session.split('_')[0]
    sessionpath = os.path.join(source, animal, session)
    
    if steps['write_trajectory']:
        pack(sessionpath)
    if steps['write_units']:
        write_units(sessionpath)
    if steps['write_spiking_metrics']:
        write_spiking_metrics(sessionpath)
    if steps['write_spatial_metrics']:
        write_spatial_metrics(sessionpath)
    if steps['write_best_mat_rot']:
        write_best_match_rotation(sessionpath)
        
    print('session %s done' % session)

session 60_SIT_2023-11-24_17-09-10 done


## Playground

In [4]:
import h5py
import os
import pandas as pd

In [6]:
source = '/home/sobolev/nevermind/Andrey/data'
session = '009266_hippoSIT_2023-04-24_16-56-55'

animal      = session.split('_')[0]
sessionpath = os.path.join(source, animal, session)
dlc_file = os.path.join(sessionpath, 'dlc.csv')

#with h5py.File(dlc_file, 'r') as f:
#    aeps = np.array(f['aeps'])
#    aeps_events = np.array(f['aeps_events'])

In [7]:
df = pd.read_csv(dlc_file)

/home/sobolev/projects/pySIT/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [8]:
df.head()

,scorer,DLC_resnet50_timeSITOct21shuffle4_700000,DLC_resnet50_timeSITOct21shuffle4_700000.1,DLC_resnet50_timeSITOct21shuffle4_700000.2,DLC_resnet50_timeSITOct21shuffle4_700000.3,DLC_resnet50_timeSITOct21shuffle4_700000.4,DLC_resnet50_timeSITOct21shuffle4_700000.5,DLC_resnet50_timeSITOct21shuffle4_700000.6,DLC_resnet50_timeSITOct21shuffle4_700000.7,DLC_resnet50_timeSITOct21shuffle4_700000.8,...,DLC_resnet50_timeSITOct21shuffle4_700000.17,DLC_resnet50_timeSITOct21shuffle4_700000.18,DLC_resnet50_timeSITOct21shuffle4_700000.19,DLC_resnet50_timeSITOct21shuffle4_700000.20,DLC_resnet50_timeSITOct21shuffle4_700000.21,DLC_resnet50_timeSITOct21shuffle4_700000.22,DLC_resnet50_timeSITOct21shuffle4_700000.23,DLC_resnet50_timeSITOct21shuffle4_700000.24,DLC_resnet50_timeSITOct21shuffle4_700000.25,DLC_resnet50_timeSITOct21shuffle4_700000.26
0,bodyparts,left_eye,left_eye,left_eye,right_eye,right_eye,right_eye,left_ear,left_ear,left_ear,...,red_dot,tail_base,tail_base,tail_base,lower_spine,lower_spine,lower_spine,neck,neck,neck
1,coords,x,y,likelihood,x,y,likelihood,x,y,likelihood,...,likelihood,x,y,likelihood,x,y,likelihood,x,y,likelihood
2,0,651.7044067382812,581.309814453125,0.9999692440032959,641.3778686523438,588.9384765625,0.9999958276748657,649.8087768554688,572.6547241210938,0.9999592304229736,...,0.9999963045120239,625.6045532226562,528.5950927734375,0.9999996423721313,629.8932495117188,544.019775390625,0.9999817609786987,637.4949951171875,568.9853515625,0.9994956254959106
3,1,651.7330322265625,581.3253173828125,0.9999746084213257,641.5486450195312,588.9435424804688,0.9999963045120239,649.8545532226562,572.6721801757812,0.999961256980896,...,0.9999960660934448,625.6072387695312,528.6365966796875,0.9999997615814209,629.9107666015625,544.2516479492188,0.9999829530715942,637.5068969726562,569.1007080078125,0.9992895126342773
4,2,651.7330932617188,581.485595703125,0.9999744892120361,641.5501098632812,588.9569702148438,0.9999963045120239,650.0103149414062,572.7637329101562,0.9999610185623169,...,0.9999960660934448,625.6411743164062,528.6522827148438,0.9999997615814209,629.9863891601562,544.2548828125,0.9999831914901733,637.69287109375,569.1029052734375,0.9992813467979431


## Read unit names from H5

In [34]:
source = '/home/sobolev/nevermind/Andrey/data'
animal = '009266'

sessions = get_sessions_list(os.path.join(source, animal), animal)
for i, session in enumerate(sessions):
    print(i, session)

0 009266_hippoSIT_2023-02-28_19-53-56
1 009266_hippoSIT_2023-03-01_17-53-11
2 009266_hippoSIT_2023-03-06_15-10-36
3 009266_hippoSIT_2023-03-06_20-43-19
4 009266_hippoSIT_2023-03-08_17-06-45
5 009266_hippoSIT_2023-03-09_09-37-07
6 009266_hippoSIT_2023-03-09_19-12-22
7 009266_hippoSIT_2023-04-12_15-49-49
8 009266_hippoSIT_2023-04-13_08-57-46
9 009266_hippoSIT_2023-04-14_09-17-34
10 009266_hippoSIT_2023-04-17_09-06-10
11 009266_hippoSIT_2023-04-17_17-04-17
12 009266_hippoSIT_2023-04-18_10-10-37
13 009266_hippoSIT_2023-04-18_17-03-10
14 009266_hippoSIT_2023-04-19_10-33-51
15 009266_hippoSIT_2023-04-19_11-21-37
16 009266_hippoSIT_2023-04-19_17-12-48
17 009266_hippoSIT_2023-04-20_08-57-39
18 009266_hippoSIT_2023-04-20_15-24-14
19 009266_hippoSIT_2023-04-21_08-43-00
20 009266_hippoSIT_2023-04-21_13-12-31
21 009266_hippoSIT_2023-04-24_10-08-11
22 009266_hippoSIT_2023-04-24_16-56-55
23 009266_hippoSIT_2023-04-25_09-02-56
24 009266_hippoSIT_2023-04-25_17-27-51
25 009266_hippoSIT_2023-04-26_08-20

In [44]:
session = sessions[8]
session = '009266_hippoSIT_2023-05-04_19-47-15'
sessionpath = os.path.join(source, animal, session)
h5_file = os.path.join(sessionpath, '%s.h5' % session)

with h5py.File(h5_file, 'r') as f:
    if 'units' in f:
        units = [unit_name for unit_name in f['units']]
    else:
        units = []

## Test sounds csv for delays

In [5]:
session = '009266_hippoSIT_2023-05-04_19-47-15'
animal = '009266'
sessionpath = os.path.join(source, animal, session)

In [7]:
sounds = np.loadtxt(os.path.join(sessionpath, 'sounds.csv'), skiprows=1, delimiter=',')

In [8]:
sounds.shape

(9599, 2)

In [1]:
import time

time.sleep(2)